In [1]:
import spacy
nlp = spacy.load('en_core_web_md')

2022-12-12 13:12:56.341484: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-12 13:12:56.341577: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-12 13:13:12.508908: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-12 13:13:12.509056: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-12 13:13:12.509188: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (u): /proc/driver/nvidia/version does not exist


In [16]:
def Preprocess(text):
    doc = nlp(text)
    filtered_token = []
    
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_token.append(token.lemma_)
        
    return " ".join(filtered_token)

In [17]:
Preprocess('"Thor eating pizza, Loki is eating pizza, Ironman ate pizza already"')

'thor eat pizza Loki eat pizza Ironman eat pizza'

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

In [5]:
import pandas as pd
data = pd.read_csv('../data/ecommerceDataset.csv')

In [73]:
df = data.groupby('Household').sample(5000)

In [74]:
df.Household.value_counts()

Books                     5000
Clothing & Accessories    5000
Electronics               5000
Household                 5000
Name: Household, dtype: int64

In [75]:
df['text'] = df.iloc[:, 1].astype(str)

In [76]:
df['preprocessed_text'] = df['text'].apply(Preprocess)

In [77]:
df['label_num'] = df['Household'].map({'Household' : 0, 'Books': 1, 'Electronics': 2, 'Clothing & Accessories': 3})

In [78]:
df['label_num'].value_counts()

1    5000
3    5000
2    5000
0    5000
Name: label_num, dtype: int64

In [79]:
x = df['preprocessed_text']
y = df['label_num']

In [80]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=22, stratify=y, test_size=.3)

In [81]:
model = Pipeline([("TF-IDF", TfidfVectorizer()),
                 ("NB", MultinomialNB())])

In [82]:
model.fit(x_train, y_train)

Pipeline(steps=[('TF-IDF', TfidfVectorizer()), ('NB', MultinomialNB())])

In [83]:
y_test_pred = model.predict(x_test)

In [84]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.91      0.94      0.93      1500
           1       0.97      0.93      0.95      1500
           2       0.95      0.94      0.94      1500
           3       0.97      0.98      0.98      1500

    accuracy                           0.95      6000
   macro avg       0.95      0.95      0.95      6000
weighted avg       0.95      0.95      0.95      6000

